In [1]:
import networkx as nx
from karateclub import BigClam
from cdlib import algorithms
from cdlib import evaluation
from cdlib import NodeClustering
import pandas as pd

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'leidenalg', 'bayanpy', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'leidenalg'}


In [2]:
df1=pd.read_csv("./dataset/ogbn_proteins/raw/edge.csv.gz")
df2=pd.read_csv("./dataset/ogbn_proteins/raw/node_species.csv.gz")

In [3]:
G = nx.from_pandas_edgelist(df1, "1", "96401")

In [4]:
species_proteins_dict = {}
for species_id in df2['3702'].unique():
    protein_ids = df2[df2['3702'] == species_id].index.tolist()
    protein_ids = [element + 1 for element in protein_ids]
    species_proteins_dict[species_id] = protein_ids
ground_truth= NodeClustering(
        list(species_proteins_dict.values()),
        G,
        "Ground_Truth",
        method_parameters=None,
        overlap=True,
    )

In [5]:
model = BigClam(dimensions=100)
model.fit(G)

In [6]:
memberships = model.get_memberships()
community_list = {}
for node, community in memberships.items():
    if community not in community_list:
        community_list[community] = [node]
    else:
        community_list[community].append(node)

# Convert the dictionary to a list of lists
coms = list(community_list.values())
communities= NodeClustering(
        coms,
        G,
        "BigClam",
        method_parameters={"dimension": 112},
        overlap=True,
    )

In [7]:
nmi=evaluation.overlapping_normalized_mutual_information_MGH(ground_truth,communities)
nmi

MatchingResult(score=0.0011773402720736137, std=None)

In [8]:
len(communities.communities)

6